In [ ]:
from selenium import webdriver
import requests
import time
import pandas as pd
from IPython.display import clear_output

C:\Users\cv002\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [ ]:
# 웹드라이드 생행
driver = webdriver.Chrome('chromedriver')

In [ ]:
# 다음 영화 사이트 들어가기
site = 'https://movie.daum.net/premovie/theater'
time.sleep(6)
driver.get(site)

In [ ]:
# ★ 스크롤 한다.
for idx in range(10):
    time.sleep(1)
    script = 'window.scrollTo(0, document.body.scrollHeight)'
    driver.execute_script(script)

In [ ]:
# 영화 전체 목록을 가져온다
a1 = driver.find_element_by_css_selector('#mainContent > div > div.box_movie > ol')
a1

<selenium.webdriver.remote.webelement.WebElement (session="0be9c8d39ba01128020dc316b8ff67a4", element="17910b3f-638b-4835-bafc-12a411e6c8ed")>

In [ ]:
# 영화 전체 목록에서 li태그들을 가져온다
li_list= a1.find_elements_by_css_selector('li')
li_list

[<selenium.webdriver.remote.webelement.WebElement (session="0be9c8d39ba01128020dc316b8ff67a4", element="af5a53eb-7f96-413f-b516-9f5559fc1138")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0be9c8d39ba01128020dc316b8ff67a4", element="bc65e7b8-a484-41b6-8542-59882565ef20")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0be9c8d39ba01128020dc316b8ff67a4", element="90587670-299a-49c6-a1c2-46ed313ae2cb")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0be9c8d39ba01128020dc316b8ff67a4", element="6fae2a49-e821-422d-9a93-a3f60ac7e3d1")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0be9c8d39ba01128020dc316b8ff67a4", element="5589e436-34b7-4fd1-9310-38263aad6c26")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0be9c8d39ba01128020dc316b8ff67a4", element="8e8bd0a7-88b4-4e4d-8325-c61d8a494faf")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0be9c8d39ba01128020dc316b8ff67a4", element="9e81d63f-6d93-436d-88f1-3a

In [ ]:
# 데이터를 담을 딕셔너리
data_dict = {
            '제목' : [],
            '평점' : [],
            '예매율' : [],
            '개봉일자' : []
}


for movie_tag in li_list:
    
    # 제목
    title_tag = movie_tag.find_element_by_css_selector('div.thumb_cont > strong > a')
    # 위에서 사용했던 태그와 중복되는 것은 지워주면 된다
    # 위에서는 li까지 사용했으니
    # #mainContent > div > div.box_movie > ol > li:nth-child(1) > div > div.thumb_cont > strong > a
    # li:nth-child(1) 까지 지워준다
    title = title_tag.text.strip()
    
    # 평점
    rating_tag = movie_tag.find_element_by_css_selector('div > div.thumb_cont > span.txt_append > span:nth-child(1) > span')
    rating = rating_tag.text.strip()
    
    # 예매율
    ticket_tag = movie_tag.find_element_by_css_selector('div > div.thumb_cont > span.txt_append > span:nth-child(2) > span')
    ticket = ticket_tag.text.strip()
    
    # 개봉일
    open_date_tag = movie_tag.find_element_by_css_selector('div > div.thumb_cont > span.txt_info > span')
    open_date = open_date_tag.text.strip()
    
    # 영화 포스터
    poster_tag = movie_tag.find_element_by_css_selector('div > div.thumb_item > div.poster_movie > img')
    src_attr = poster_tag.get_attribute('src')
    
    # 영화의 제목을 파일명으로 사용할 것이기 때문에 
    # os에서 거부하는 파일명을 정제한다.
    
    char_list = ['\\', '/',':','*','"','?','<','>','|','\n']
    
    file_name = title
    
    for c1 in char_list:
        file_name = file_name.replace(c1, ' ')
    
    # 이미지 데이터를 내려받는다.
    img_res = requests.get(src_attr)
    
    # 저장한다.
    with open(f'poster/{file_name}.jpg', 'wb') as fp:
        fp.write(img_res.content)
    
#     print(src_attr)
#     print(title)
#     print(f'평점 : {rating}')
#     print(ticket[:-1])
#     print(open_date)
#     print('-'*35)
    
    # 데이터를 담는다
    data_dict['제목'].append(title)
    data_dict['평점'].append(rating)
    data_dict['예매율'].append(ticket)
    data_dict['개봉일자'].append(open_date)
    
# 데이터 프레임을 생성
df1 = pd.DataFrame(data_dict)

In [ ]:
df1

,제목,평점,예매율,개봉일자
0,인질,8.5,37.6%,21.08.18
1,싱크홀,6.8,20.8%,21.08.11
2,올드,8.5,12.7%,21.08.18
3,모가디슈,8.8,11.3%,21.07.28
4,프리 가이,8.3,2.8%,21.08.11
5,남색대문,8.8,2.8%,21.08.18
6,퍼피 구조대 더 무비,10.0,1.6%,21.08.18
7,하이큐!! 땅 VS 하늘,7.5,1.3%,21.08.12
8,더 수어사이드 스쿼드,6.8,0.7%,21.08.04
9,킬러의 보디가드 2: 킬러의 와이프,5.9,0.5%,21.08.04


In [ ]:
file_name

'나는 나대로 혼자서 간다'

In [ ]:
df1.to_csv('daum_moive.csv',index=False,encoding='utf-8-sig')
print('저장완료')